In [4]:
import av
import torch
import numpy as np

def read_video_av(path, max_frames=32):
    container = av.open(path)
    frames = []
    for frame in container.decode(video=0):
        img = frame.to_ndarray(format="rgb24")
        frames.append(img)
        if len(frames) >= max_frames:
            break
    frames = np.stack(frames)  # (T, H, W, C)
    frames = torch.tensor(frames).permute(0, 3, 1, 2) / 255.0  # (T, C, H, W)
    return frames

video_tensor = read_video_av("../20bn-something-something-v2/100.webm", max_frames=16)
print(video_tensor.shape)

torch.Size([16, 3, 240, 427])


In [12]:
import av

def get_total_frames(path):
    container = av.open(path)
    total = 0
    for frame in container.decode(video=0):
        total += 1
    return total
for i in range(1,50):
    frames_count = get_total_frames(f"../20bn-something-something-v2/{i}.webm")
    print("Total frames:", frames_count)

Total frames: 47
Total frames: 48
Total frames: 57
Total frames: 42
Total frames: 24
Total frames: 43
Total frames: 31
Total frames: 36
Total frames: 36
Total frames: 44
Total frames: 59
Total frames: 47
Total frames: 44
Total frames: 45
Total frames: 73
Total frames: 54
Total frames: 62
Total frames: 38
Total frames: 46
Total frames: 31
Total frames: 37
Total frames: 58
Total frames: 58
Total frames: 30
Total frames: 37
Total frames: 54
Total frames: 39
Total frames: 54
Total frames: 60
Total frames: 45
Total frames: 48
Total frames: 29
Total frames: 42
Total frames: 50
Total frames: 43
Total frames: 59
Total frames: 47
Total frames: 28
Total frames: 62
Total frames: 38
Total frames: 28
Total frames: 69
Total frames: 53
Total frames: 40
Total frames: 39
Total frames: 66
Total frames: 58
Total frames: 40
Total frames: 40


## move_object

In [ ]:
 "Moving away from something with your camera":"32",
 "Moving part of something":"33",
 "Moving something across a surface until it falls down":"34",
 "Moving something across a surface without it falling down":"35",
 "Moving something and something away from each other":"36",
 "Moving something and something closer to each other":"37",
 "Moving something and something so they collide with each other":"38",
 "Moving something and something so they pass each other":"39",
 "Moving something away from something":"40",
 "Moving something away from the camera":"41",
 "Moving something closer to something":"42",
 "Moving something down":"43",
 "Moving something towards the camera":"44",
 "Moving something up":"45",

 "Pushing something from left to right":"93",
 "Pushing something from right to left":"94",
 "Pushing something off of something":"95",
 "Pushing something onto something":"96",
 "Pushing something so it spins":"97",
 "Pushing something so that it almost falls off but doesn't":"98",
 "Pushing something so that it falls off the table":"99",
 "Pushing something so that it slightly moves":"100",
 "Pushing something with something":"101",

## drop_object

In [ ]:
from pathlib import Path
import csv
import json

label_dir = Path('..') / 'labels'
drop_dir = label_dir / 'drop_object'
drop_dir.mkdir(parents=True, exist_ok=True)
drop_templates = [
    "Dropping [something] behind [something]",
    "Dropping [something] in front of [something]",
    "Dropping [something] into [something]",
    "Dropping [something] next to [something]",
    "Dropping [something] onto [something]",
    "Moving [something] across a surface until it falls down",
    "Putting [something] on the edge of [something] so it is not supported and falls down",
    "Putting [something] onto [something else that cannot support it] so it falls down"
]

def normalize_template(template: str) -> str:
    return template.replace('[', '').replace(']', '')

normalized_templates = {normalize_template(t) for t in drop_templates}
answers_path = label_dir / 'test-answers.csv'
test_answer_ids = set()
if answers_path.exists():
    with answers_path.open(newline='', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=';')
        for row in reader:
            if len(row) != 2:
                continue
            clip_id, template_text = row[0].strip(), row[1].strip()
            if template_text in normalized_templates:
                test_answer_ids.add(clip_id)
else:
    print(f'{answers_path} 缺失，测试集无法筛选')

split_files = {
    'train': label_dir / 'train.json',
    'validation': label_dir / 'validation.json',
    'test': label_dir / 'test.json'
}
name_map = {'train': 'train', 'validation': 'val', 'test': 'test'}
stats = {}
for split, file_path in split_files.items():
    if not file_path.exists():
        print(f'{file_path} 缺失，跳过 {split}')
        continue
    entries = json.loads(file_path.read_text())
    if split == 'test':
        matched = [e for e in entries if str(e.get('id')) in test_answer_ids]
    else:
        matched = [e for e in entries if e.get('template') in drop_templates]
    stats[split] = len(matched)
    out_path = drop_dir / f"{name_map[split]}_drop_object.json"
    out_path.write_text(json.dumps(matched, ensure_ascii=False, indent=2))
    print(f"{split}: {len(matched)} entries -> {out_path}")
print('Summary:', stats)



## cover_object

In [ ]:
from pathlib import Path
import csv
import json

label_dir = Path('..') / 'labels'
cover_dir = label_dir / 'cover_object'
cover_dir.mkdir(parents=True, exist_ok=True)
cover_templates = [
    "Covering [something] with [something]",
    "Putting [number of] [something] onto [something]",
    "Putting [something] and [something] on the table",
    "Putting [something] on a surface",
    "Putting [something] onto a slanted surface but it doesn't glide down",
    "Putting [something] onto [something]",
    "Putting [something similar to other things that are already on the table]",
    "Putting [something] that can't roll onto a slanted surface, so it slides down",
    "Putting [something] that can't roll onto a slanted surface, so it stays where it is",
    "Putting [something] upright on the table",
    "Putting [something], [something] and [something] on the table"
]

def normalize_template(template: str) -> str:
    return template.replace('[', '').replace(']', '')

normalized_templates = {normalize_template(t) for t in cover_templates}
answers_path = label_dir / 'test-answers.csv'
test_answer_ids = set()
if answers_path.exists():
    with answers_path.open(newline='', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=';')
        for row in reader:
            if len(row) != 2:
                continue
            clip_id, template_text = row[0].strip(), row[1].strip()
            if template_text in normalized_templates:
                test_answer_ids.add(clip_id)
else:
    print(f'{answers_path} 缺失，测试集无法筛选')

split_files = {
    'train': label_dir / 'train.json',
    'validation': label_dir / 'validation.json',
    'test': label_dir / 'test.json'
}
name_map = {'train': 'train', 'validation': 'val', 'test': 'test'}
stats = {}
for split, file_path in split_files.items():
    if not file_path.exists():
        print(f'{file_path} 缺失，跳过 {split}')
        continue
    entries = json.loads(file_path.read_text())
    if split == 'test':
        matched = [e for e in entries if str(e.get('id')) in test_answer_ids]
    else:
        matched = [e for e in entries if e.get('template') in cover_templates]
    stats[split] = len(matched)
    out_path = cover_dir / f"{name_map[split]}_cover_object.json"
    out_path.write_text(json.dumps(matched, ensure_ascii=False, indent=2))
    print(f"{split}: {len(matched)} entries -> {out_path}")
print('Summary:', stats)



## 验证分割有效性

In [ ]:
from pathlib import Path  # 路径处理工具
import csv  # 读取test-answers
import json  # JSON读写
label_dir = Path('..') / 'labels'  # 标签目录
move_dir = label_dir / 'move_object'  # Move_object输出目录
move_dir.mkdir(parents=True, exist_ok=True)  # 确保输出目录存在
move_templates = [  # 手动列出需要匹配的模板
     "Pushing [something] from left to right",
]  # Move_object模板列表
def normalize_template(template: str) -> str:
    return template.replace('[', '').replace(']', '')  # 去掉[]以对齐test-answers里的文案
normalized_templates = {normalize_template(t) for t in move_templates}
split_files = {  # train/validation/test标签路径
    'train': label_dir / 'train.json',
    'validation': label_dir / 'validation.json',
    'test': label_dir / 'test.json'
}  # 按划分存储路径
output_files = {  # 对应输出文件
    'train': move_dir / 'train_move_object.json',
    'validation': move_dir / 'val_move_object.json',
    'test': move_dir / 'test_move_object.json'
}
test_answer_ids = set()  # test-answers筛出的ID集合
answers_path = label_dir / 'test-answers.csv'
if answers_path.exists():
    with answers_path.open(newline='', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=';')
        for row in reader:
            if len(row) != 2:
                continue
            clip_id, template_text = row[0].strip(), row[1].strip()
            if template_text in normalized_templates:
                test_answer_ids.add(clip_id)
else:
    print(f'{answers_path} 缺失，测试集无法筛选')
stats = {}  # 存放统计数量
for split, file_path in split_files.items():  # 遍历所有划分
    if not file_path.exists():  # 如果文件不存在
        print(f'{file_path} 缺失，跳过 {split}')  # 打印提醒
        continue  # 跳过本划分
    entries = json.loads(file_path.read_text())  # 读取标签列表
    if split == 'test':
        matched = [e for e in entries if str(e.get('id')) in test_answer_ids]
    else:
        matched = [e for e in entries if e.get('template') in move_templates]
    stats[split] = len(matched)  # 记录数量
    out_path = output_files[split]  # 输出路径
    out_path.write_text(json.dumps(matched, ensure_ascii=False, indent=2))  # 写入文件
    print(f'{split}: {len(matched)} entries -> {out_path}')  # 打印进度
print('Summary:', stats)  # 打印汇总
print('Total:',stats['train'] + stats['validation'] + stats['test']) #看总数是否和预期一致，一致则说明没有遗漏


train: 2949 entries -> ../labels/move_object/train_move_object.json
validation: 250 entries -> ../labels/move_object/val_move_object.json
test: 243 entries -> ../labels/move_object/test_move_object.json
Summary: {'train': 2949, 'validation': 250, 'test': 243}
Total: 3442


In [9]:
from pathlib import Path  # 路径处理工具
import csv  # 读取test-answers
import json  # JSON读写
label_dir = Path('..') / 'labels'  # 标签目录
move_dir = label_dir / 'move_object'  # Move_object输出目录
move_dir.mkdir(parents=True, exist_ok=True)  # 确保输出目录存在
move_templates = [  # 手动列出需要匹配的模板
    "Moving away from [something] with your camera",
    "Moving [something] across a surface until it falls down",
    "Moving [something] across a surface without it falling down",
    "Moving [something] and [something] away from each other",
    "Moving [something] and [something] closer to each other",
    "Moving [something] and [something] so they collide with each other",
    "Moving [something] and [something] so they pass each other",
    "Moving [something] away from [something]",
    "Moving [something] away from the camera",
    "Moving [something] closer to [something]",
    "Moving [something] down",
    "Moving [something] towards the camera",
    "Moving [something] up",
    "Pulling [something] from behind of [something]",
    "Pulling [something] from left to right",
    "Pulling [something] from right to left",
    "Pulling [something] onto [something]",
    "Pulling [something] out of [something]",
    "Pushing [something] from left to right",
    "Pushing [something] from right to left",
    "Pushing [something] off of [something]",
    "Pushing [something] onto [something]",
    "Pushing [something] so it spins",
    "Pushing [something] so that it almost falls off but doesn't",
    "Pushing [something] so that it falls off the table",
    "Pushing [something] so that it slightly moves",
    "Pushing [something] with [something]"
]  # Move_object模板列表
def normalize_template(template: str) -> str:
    return template.replace('[', '').replace(']', '')  # 去掉[]以对齐test-answers里的文案
normalized_templates = {normalize_template(t) for t in move_templates}
split_files = {  # train/validation/test标签路径
    'train': label_dir / 'train.json',
    'validation': label_dir / 'validation.json',
    'test': label_dir / 'test.json'
}  # 按划分存储路径
output_files = {  # 对应输出文件
    'train': move_dir / 'train_move_object.json',
    'validation': move_dir / 'val_move_object.json',
    'test': move_dir / 'test_move_object.json'
}
test_answer_ids = set()  # test-answers筛出的ID集合
answers_path = label_dir / 'test-answers.csv'
if answers_path.exists():
    with answers_path.open(newline='', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=';')
        for row in reader:
            if len(row) != 2:
                continue
            clip_id, template_text = row[0].strip(), row[1].strip()
            if template_text in normalized_templates:
                test_answer_ids.add(clip_id)
else:
    print(f'{answers_path} 缺失，测试集无法筛选')
stats = {}  # 存放统计数量
for split, file_path in split_files.items():  # 遍历所有划分
    if not file_path.exists():  # 如果文件不存在
        print(f'{file_path} 缺失，跳过 {split}')  # 打印提醒
        continue  # 跳过本划分
    entries = json.loads(file_path.read_text())  # 读取标签列表
    if split == 'test':
        matched = [e for e in entries if str(e.get('id')) in test_answer_ids]
    else:
        matched = [e for e in entries if e.get('template') in move_templates]
    stats[split] = len(matched)  # 记录数量
    out_path = output_files[split]  # 输出路径
    out_path.write_text(json.dumps(matched, ensure_ascii=False, indent=2))  # 写入文件
    print(f'{split}: {len(matched)} entries -> {out_path}')  # 打印进度
print('Summary:', stats)  # 打印汇总


train: 33493 entries -> ../labels/move_object/train_move_object.json
validation: 4093 entries -> ../labels/move_object/val_move_object.json
test: 4427 entries -> ../labels/move_object/test_move_object.json
Summary: {'train': 33493, 'validation': 4093, 'test': 4427}
